In [49]:
import numpy as np
import itertools
import random

In [50]:
p_running = 0.9
p_failure = 1 - p_running
path = '/home/dkabe/Model_brainstorming/SAA_Analysis/'

In [52]:
provinces = [3, 6]
plants = 6
dc = 4
a5 = list(itertools.product([1, 0], repeat = provinces[0]))
a6 = list(itertools.product([1, 0], repeat = provinces[1]))
b = list(itertools.product([1, 0], repeat = dc))
mp_loc_encoded = [[0, 0, 1, 1, 2, 2],
                  [0, 1, 2, 3, 4, 5]]

In [53]:
province_disruptions = [[x,y] for x in a6 for y in b]
num_Scenarios = len(province_disruptions)
plant_failure = [[None]*plants for x in range(2**provinces[1])]
for k in range(len(plant_failure)):
    for i in range(plants):
        ind = mp_loc_encoded[1][i]
        plant_failure[k][i] = a6[k][ind]
Scen = [[x,y] for x in plant_failure for y in b]
p_scen = []
for s in range(len(province_disruptions)):
    p_i = (p_running**(np.sum(province_disruptions[s][0]) + np.sum(province_disruptions[s][1])))*(p_failure**(provinces[1] + dc - (np.sum(province_disruptions[s][0]) + np.sum(province_disruptions[s][1]))))
    p_scen.append(p_i)

In [54]:
N = [50, 100, 150, 200, 250, 300]
batches = 30

In [55]:
random.seed(1)
for num_scen in N:
    for batch in range(batches):
        indices = random.sample(range(len(Scen)), num_scen)
        sample_scenarios = [Scen[index] for index in indices]
        p_sample_scenarios = [p_scen[index] for index in indices]
        p_factor = 1/np.sum(p_sample_scenarios)
        p_sample_scenarios = list(map(lambda x: x*p_factor, p_sample_scenarios))  
        with open(path + 'Scenarios/' + str(num_scen) + '_' + str(batch) + '.txt', 'w') as f:
            for item in sample_scenarios:
                f.write("%s\n" % item)
        f.close()
        np.savetxt(path + 'Scen_probabilities/' + 'p_scen_' + str(num_scen) + '_' + str(batch) + '.txt', p_sample_scenarios)


In [60]:
counter = 0 
for num_scen in N:
    for batch in range(batches):
        probs = np.loadtxt(path + 'Scen_probabilities/' + 'p_scen_' + str(num_scen) + '_' + str(batch) + '.txt')
        counter += np.sum(probs)

In [61]:
counter

180.0

In [40]:
p1_mean = 137000
p2_mean = 135000
p3_mean = 133000

p1_sd = 2650
p2_sd = 8125
p3_sd = 4000

distribution_mat = [[p1_mean, p1_sd],
                  [p2_mean, p2_sd],
                  [p3_mean, p3_sd]]

Products = 3
Market = 29

In [62]:
np.random.seed(1)

for num_scen in N:
    for batch in range(batches):
        demand = np.zeros((num_scen, Products, Market))
        for s in range(num_scen):
            demand[s][0] = np.round(np.random.normal(distribution_mat[0][0], distribution_mat[0][1], size = 29))
            demand[s][1] = np.round(np.random.normal(distribution_mat[1][0], distribution_mat[1][1], size = 29))
            demand[s][2] = np.round(np.random.normal(distribution_mat[2][0], distribution_mat[2][1], size = 29))
            np.savetxt(path + 'Scen_demand/' + 'demand_' + str(num_scen) + '_' + str(batch) + '.txt', demand.reshape((num_scen*Products, Market)))


In [73]:
d = np.loadtxt(path + 'Scen_demand/' + 'demand_' + '50' + '_' + str(3) + '.txt').reshape((50, Products, Market))